# Claude Memory on Amazon Bedrock

This notebook demonstrates Claude's native memory feature across different chat sessions on Amazon Bedrock.

## Key Concepts:
- Starting with Claude Sonnet 4.5, and include support for previous model releases, Claude can store persistent memories using its built-in memory tool
- Memories persist across different chat sessions
- The client only needs to handle tool use/result exchange
- Claude directs the memory storage and retrieval, and storage and retrieval is done on the client side

## Setup: Minimal Client Code

In [ ]:
import boto3
import json
from datetime import datetime
from botocore.config import Config

config = Config(
   retries = {
      'total_max_attempts': 1000,
      'mode': 'standard'
   }
)

# AWS session setup
session = boto3.session.Session()
bedrock_rt = session.client(
    service_name='bedrock-runtime',
    config=config
)

# Simple memory storage for handling tool results
memory_store = {}

def handle_memory_tool(tool_input):
    """Minimal handler for Claude Sonnet 4.5's memory tool requests"""
    command = tool_input.get('command')
    path = tool_input.get('path', '')
    
    if command == 'view':
        if path == '/memories':
            return {'memories': list(memory_store.values())}
        elif path.startswith('/memories/'):
            key = path.split('/')[-1]
            return {'memory': memory_store.get(key, {'error': 'not found'})}
    
    elif command == 'create':
        key = path.split('/')[-1] if '/' in path else f"mem_{len(memory_store)}"
        memory_store[key] = {
            'path': path,
            'content': tool_input.get('file_text', ''),
            'created': datetime.now().isoformat()
        }
        return {'success': True, 'created': key}
    
    return {'status': 'handled', 'command': command}

print("✅ Minimal client setup complete")

✅ Minimal client setup complete


## Session 1: Ask Claude Sonnet 4.5 to Remember Information

First, we'll have a conversation where Claude Sonnet 4.5 stores memories about us.

In [2]:
def chat_with_claude(message, conversation_history=None):
    """Send a message to Claude Sonnet 4.5 and handle any tool use"""
    messages = conversation_history or []
    
    # Add user message
    messages.append({
        "role": "user",
        "content": [{"type": "text", "text": message}]
    })
    
    print(f"👤 User: {message}\n")
    
    # Keep handling tool uses until Claude Sonnet 4.5 stops requesting them
    max_iterations = 5  # Safety limit
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        
        # Prepare request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "anthropic_beta": ["context-management-2025-06-27"],
            "tools": [{
                "type": "memory_20250818",
                "name": "memory"
            }],
            "max_tokens": 4000,
            "messages": messages,
        }
        
        # Send to Claude Sonnet 4.5
        response = bedrock_rt.invoke_model(
            modelId="us.anthropic.claude-sonnet-4-5-20250929-v1:0",
            body=json.dumps(body)
        )
        result = json.loads(response["body"].read())
        
        # Track if we need to handle tool use
        has_tool_use = False
        tool_uses = []
        
        # Process response
        for content in result.get('content', []):
            if content['type'] == 'text':
                print(f"🤖 Claude: {content['text']}\n")
            
            elif content['type'] == 'tool_use':
                has_tool_use = True
                tool_uses.append(content)
                print(f"🔧 Claude uses memory tool: {content['input'].get('command', 'unknown')}")
                print(f"   Path: {content['input'].get('path', 'N/A')}")
                if content['input'].get('file_text'):
                    print(f"   Creating memory file...\n")
        
        # Add Claude's response to conversation
        messages.append({
            "role": "assistant",
            "content": result['content']
        })
        
        # If no tool use, we're done
        if not has_tool_use:
            break
        
        # Handle all tool uses
        tool_results = []
        for tool_use in tool_uses:
            tool_result = handle_memory_tool(tool_use['input'])
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_use['id'],
                "content": json.dumps(tool_result)
            })
        
        # Add tool results to conversation
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Continue loop to get Claude's next response
    
    return messages

print("✅ Chat function ready")

✅ Chat function ready


In [3]:
print("=" * 60)
print("SESSION 1: Teaching Claude Sonnet 4.5 About Me")
print("=" * 60)
print()

# First conversation - Claude Sonnet 4.5 will remember this
conversation1 = chat_with_claude(
    "Hi Claude! Please remember these important things about me: "
    "1) I'm a Python developer working on machine learning projects, "
    "2) I prefer using VSCode with dark theme, "
    "3) My favorite Python libraries are numpy, pandas, and scikit-learn, "
    "4) I'm currently learning about LLM memory systems like yours!"
)

SESSION 1: Teaching Claude Sonnet 4.5 About Me

👤 User: Hi Claude! Please remember these important things about me: 1) I'm a Python developer working on machine learning projects, 2) I prefer using VSCode with dark theme, 3) My favorite Python libraries are numpy, pandas, and scikit-learn, 4) I'm currently learning about LLM memory systems like yours!

🔧 Claude uses memory tool: view
   Path: /memories
🔧 Claude uses memory tool: create
   Path: /memories/user_profile.txt
   Creating memory file...

🤖 Claude: 

Perfect! I've saved all that important information about you to my memory. Here's what I've recorded:

✅ **Professional**: Python developer working on ML projects
✅ **Dev Environment**: VSCode with dark theme
✅ **Favorite Libraries**: numpy, pandas, and scikit-learn
✅ **Current Learning**: LLM memory systems

I'll remember these details for our future conversations! It's great that you're learning about LLM memory systems - you're literally experiencing one right now. I use a fil

In [4]:
# Check what was stored locally
print("📦 Local memory store after Session 1:")
print("=" * 40)
for key, value in memory_store.items():
    print(f"Key: {key}")
    print(f"Path: {value.get('path', 'N/A')}")
    if value.get('content'):
        print(f"Content preview: {value['content'][:200]}...")
    print("-" * 40)

📦 Local memory store after Session 1:
Key: user_profile.txt
Path: /memories/user_profile.txt
Content preview: USER PROFILE

Professional Background:
- Python developer
- Works on machine learning projects

Development Environment:
- Preferred IDE: VSCode
- Theme preference: Dark theme

Favorite P...
----------------------------------------


## Session 2: New Chat - Claude Sonnet 4.5 Recalls Information

Now we start a **completely new conversation** without any prior context, and ask Claude Sonnet 4.5 what it knows about us.

In [5]:
print("=" * 60)
print("SESSION 2: New Chat - Testing Memory Recall")
print("=" * 60)
print("\n🔄 Starting fresh conversation (no prior context)\n")

# Completely new conversation - Claude Sonnet 4.5 should remember from before
conversation2 = chat_with_claude(
    "What do you remember about me and my interests?"
)

SESSION 2: New Chat - Testing Memory Recall

🔄 Starting fresh conversation (no prior context)

👤 User: What do you remember about me and my interests?

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on my memory, here's what I know about you:

**Professional Background:**
- You're a Python developer who works on machine learning projects

**Development Preferences:**
- You use VSCode as your preferred IDE
- You like dark themes

**Favorite Tools:**
- Your go-to Python libraries are numpy, pandas, and scikit-learn

**Current Interests:**
- You're learning about LLM memory systems
- You're interested in understanding how memory works in AI assistants

Is there anything you'd like to add or update about yourself?



In [7]:
# Another query in new session
print("\n" + "=" * 60)
print("Testing specific recall...")
print("=" * 60 + "\n")

conversation3 = chat_with_claude(
    "Based on what you know about me, what Python library would you recommend "
    "for a data visualization project?"
)


Testing specific recall...

👤 User: Based on what you know about me, what Python library would you recommend for a data visualization project?

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on what I know about you, I'd recommend **Matplotlib** or **Seaborn** for your data visualization project. Here's why:

**Primary recommendation: Matplotlib**
- It integrates seamlessly with **NumPy** and **Pandas** (your favorite libraries!)
- Essential for ML projects - perfect for visualizing model performance, training curves, feature distributions, etc.
- Very versatile and powerful for scientific/statistical visualizations
- Industry standard in the ML/data science community

**Secondary recommendation: Seaborn**
- Built on top of Matplotlib
- Works beautifully with Pandas DataFrames
- Great for statistical visualizations common in ML (correlation matrices, distributions, pair plots)
- More aesthetically pleasing defaults out of the box
- Less code for complex statistic

## Session 3: Memory-Based Assistance

Now let's demonstrate Claude Sonnet 4.5 using stored memories to provide personalized help.

In [8]:
print("=" * 60)
print("SESSION 3: Personalized Code Generation")
print("=" * 60)
print("\n🎯 Claude Sonnet 4.5 will use memories to generate personalized code\n")

conversation4 = chat_with_claude(
    "Can you write me a Python script template that matches my preferences? "
    "Include imports for my favorite libraries and add appropriate comments."
)

SESSION 3: Personalized Code Generation

🎯 Claude Sonnet 4.5 will use memories to generate personalized code

👤 User: Can you write me a Python script template that matches my preferences? Include imports for my favorite libraries and add appropriate comments.

🤖 Claude: I'll check my memory first to see if I have any information about your preferences and favorite libraries.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Perfect! I found your preferences. Let me create a Python script template that matches your profile with your favorite libraries (numpy, pandas, and scikit-learn).

```python
#!/usr/bin/env python3
"""
Script Template

Description:
    [Add your script description here]

Author: [Your Name]
Date: [Date]
"""

# Standard library imports
import os
import sys
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any

# Data manipulation and numerical computing
import numpy as np
import pandas as pd

# Machine learning
from sklearn.model

## Demonstrating Memory Updates

In [9]:
print("=" * 60)
print("SESSION 4: Updating Memories")
print("=" * 60)
print()

conversation5 = chat_with_claude(
    "Update your memory about me: I've recently started using PyTorch for deep learning "
    "and I'm also interested in transformer models. Also, I now prefer using Jupyter Lab "
    "instead of just VSCode."
)

SESSION 4: Updating Memories

👤 User: Update your memory about me: I've recently started using PyTorch for deep learning and I'm also interested in transformer models. Also, I now prefer using Jupyter Lab instead of just VSCode.

🔧 Claude uses memory tool: view
   Path: /memories
🔧 Claude uses memory tool: str_replace
   Path: /memories/user_profile.txt
🤖 Claude: 

Perfect! I've updated my memory about you. Here's what I've recorded:

**New additions:**
- You've recently started using **PyTorch** for deep learning
- You're interested in **transformer models**
- You now prefer using **Jupyter Lab** instead of just VSCode

These updates have been added to your profile, so I'll remember your focus on PyTorch and transformers, as well as your IDE preference for future conversations!



In [10]:
# Verify the update worked
print("\n" + "=" * 60)
print("Verifying memory update...")
print("=" * 60 + "\n")

conversation6 = chat_with_claude(
    "What do you know about my deep learning interests?"
)


Verifying memory update...

👤 User: What do you know about my deep learning interests?

🤖 Claude: I'll check my memory to see what I know about your deep learning interests.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on my memory, I can see that you're a Python developer who works on machine learning projects, but I don't have specific information about **deep learning interests** recorded yet. 

What I do know is:
- You work on machine learning projects
- Your favorite Python libraries include numpy, pandas, and scikit-learn
- You're currently learning about LLM memory systems and how memory works in AI assistants
- You use VSCode with a dark theme

However, I don't have detailed information about specific deep learning topics, frameworks (like PyTorch or TensorFlow), architectures, or applications that interest you. Would you like to tell me more about your deep learning interests so I can keep track of them?



## Memory Analytics

In [11]:
print("📊 Memory Usage Statistics")
print("=" * 40)
print(f"Total memory operations: {len(memory_store)}")
print(f"Memory keys: {list(memory_store.keys())}")
print()

# Show timeline of memory creation
print("📅 Memory Timeline:")
for key, value in memory_store.items():
    created = value.get('created', 'Unknown')
    print(f"  {created[:19]} - {key}")

📊 Memory Usage Statistics
Total memory operations: 1
Memory keys: ['user_profile.txt']

📅 Memory Timeline:
  2025-09-29T10:35:54 - user_profile.txt


## Advanced: Cross-Session Context Test

In [12]:
def test_memory_persistence():
    """Test that Claude Sonnet 4.5 remembers across completely independent sessions"""
    
    print("🧪 Testing memory persistence across sessions\n")
    
    # Session A: Store a secret code
    print("Session A: Storing information...")
    chat_with_claude("Remember this secret project code: ALPHA-2024-ML")
    
    print("\n" + "=" * 40 + "\n")
    
    # Session B: Completely new, retrieve the code
    print("Session B: New session, retrieving information...")
    chat_with_claude("What was the secret project code I told you about?")

test_memory_persistence()

🧪 Testing memory persistence across sessions

Session A: Storing information...
👤 User: Remember this secret project code: ALPHA-2024-ML

🔧 Claude uses memory tool: view
   Path: /memories
🔧 Claude uses memory tool: create
   Path: /memories/secret_codes.txt
   Creating memory file...

🤖 Claude: 

I've recorded the secret project code **ALPHA-2024-ML** in my memory. I'll remember this for you!



Session B: New session, retrieving information...
👤 User: What was the secret project code I told you about?

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on my memory, the secret project code you told me about is **ALPHA-2024-ML**.



## Practical Example: Building a Personalized Assistant

In [13]:
print("=" * 60)
print("PRACTICAL DEMO: Personalized ML Project Setup")
print("=" * 60)
print()

# Claude Sonnet 4.5 will use all stored memories to create a personalized project
conversation_final = chat_with_claude(
    "Based on everything you know about me, create a project structure "
    "for a new machine learning project. Include a requirements.txt with "
    "my preferred libraries, a main.py template with my coding style, "
    "and a README that reflects my interests."
)

PRACTICAL DEMO: Personalized ML Project Setup

👤 User: Based on everything you know about me, create a project structure for a new machine learning project. Include a requirements.txt with my preferred libraries, a main.py template with my coding style, and a README that reflects my interests.

🤖 Claude: I'll check my memory to see what I know about you, then create a personalized project structure.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Great! Based on your profile, I can see you're a Python developer working on ML projects who prefers numpy, pandas, and scikit-learn. You use VSCode with a dark theme and are interested in LLM memory systems. Let me create a personalized ML project structure for you.

🔧 Claude uses memory tool: str_replace
   Path: /memories/user_profile.txt
🤖 Claude: 

Now, let me create your personalized ML project structure:

## Project Structure Created:

### 1. **requirements.txt** - Your preferred libraries
```txt
# Core ML Libraries
nump

In [14]:
# Final memory state
print("\n" + "=" * 60)
print("📦 Final Memory Store State")
print("=" * 60)
print(f"\nTotal memories created: {len(memory_store)}")
print("\nMemory contents:")
for key, value in memory_store.items():
    print(f"\n[{key}]")
    content = value.get('content', '')
    if content:
        # Show first few lines
        lines = content.split('\n')[:3]
        for line in lines:
            if line.strip():
                print(f"  {line[:70]}..." if len(line) > 70 else f"  {line}")


📦 Final Memory Store State

Total memories created: 2

Memory contents:

[user_profile.txt]
  USER PROFILE

[secret_codes.txt]
  SECRET PROJECT CODES


## Summary

This notebook demonstrated:

1. **Claude Sonnet 4.5's Native Memory** - The model has built-in memory capabilities
2. **Persistence** - Memories persist across completely new chat sessions
3. **Minimal Client Code** - Only need to handle tool use/result exchange
4. **Personalization** - Claude Sonnet 4.5 uses memories to provide tailored responses
5. **Dynamic Updates** - Memories can be updated as information changes

The key insight: Claude Sonnet 4.5 manages its own memory system - the client just needs to provide a simple storage layer for the tool results.